In [1]:
import openpiv.tools
import openpiv.process
import openpiv.scaling
import numpy as np
import glob
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
filelist = sorted(glob.glob('shifted/*.tif'))
print filelist[0],filelist[1]
# Read the pair of images
img_a  = openpiv.tools.imread( filelist[0] )
img_b  = openpiv.tools.imread( filelist[1] )

IndexError: list index out of range

In [ ]:

# Show the image in greyscale
imshow(np.c_[img_a,img_b],cmap=cm.gray) 

In [ ]:
window_size = 32 # pixels
overlap = 16 # pixels, i.e. 50%
dt = 1./15 # 15 Hz laser and CCD recording
# process PIV
u, v, sig2noise = \
openpiv.process.extended_search_area_piv( \
                                         img_a, img_b, \
                                         window_size=window_size,\
                                         overlap = overlap, \
                                         dt = dt, \
                                         search_area_size=2*window_size, \
                                         sig2noise_method='peak2peak' )

# get the central points of the interrogation windows
x, y = openpiv.process.get_coordinates( image_size = img_a.shape, \
                                       window_size = window_size, \
                                       overlap  = overlap )

In [ ]:
# peak to peak ratio to mask errorneous vectors is about 1.1 - 1.3
u, v, mask = openpiv.validation.sig2noise_val( u, v, sig2noise, threshold = 1.1 )

In [ ]:
# Replace outliers (masked in the previous step) using the local mean filter:
u, v = openpiv.filters.replace_outliers( u, v, method='localmean', max_iter=10, kernel_size=2)

In [ ]:
# rescale the data from pixel/dt into mm/sec
x, y, u, v = openpiv.scaling.uniform(x, y, u, v, scaling_factor = 1.0 )

In [ ]:
# save the result if you need or show it in a plot (see below):
openpiv.tools.save(x, y, u, v, mask, filelist[0]+'.txt' )
# openpiv.tools.display_vector_field(filelist[0]+'.txt', scale=5000, width=0.0025)

In [ ]:
# Show the vectof flow field using vertical velocity for the color mapping
figure(figsize=(10,10))
ax = axes()
quiver(x,y,u,v,u**2+v**2,scale=3000)
axis('tight')
ax.set_aspect(.9)
# f = gcf()
# f.set_size_inches(10,10)
colorbar(orientation='horizontal')
xlabel('x [pixel]')
ylabel('y [pixel]')
